# Observations

This tutorial will help you access the observation, through the DPClimat API, which provides access to the data of the Météo France observation network.

The main object of interest is the DPClim, which is initialized with your application ID and an observation frequency.


In [ ]:
from meteole import DPClim

APP_ID = "<TO-BE-SET>"

# Initialize the client with the observation frequency: "6m","hourly","daily","decade","monthly"
client = DPClim(application_id=APP_ID, frequency="daily")

Observations are requested with the fetch_data method for a particular station and a particular date range. The station is identified by its code. We explore later methods to help with identifying the station code. The start and end dates are specified as ISO format  strings, or as datetime objects. data is returned as a pandas dataframe, with one row per observation and columns the differente parameters of the observation, such as temperature, humidity, etc. The exact parameters available depend on the station and the frequency of the observations.

In [ ]:
station_id = "01014002"
data = client.fetch_data(station_id, "2024-01-01T00:00:00Z", "2024-01-05T00:00:00Z")
print(data.head())

# equivalent version with datetime
# from datetime import datetime
# data = client.fetch_data(station_id, datetime(2024,1,1), datetime(2024,1,5))

# A two step process

One thing to keep in mind is the request process. When you call fetch_data, the client sends a request to the DPClimat API. The API then processes the request, which may take some time depending on the size of the data requested and the current load on the server. fetch_data waits a 5 seconds by default (controlled by wait_for_file parameter) before requesting the processed data. If the data is not available, it is treated as a "normal" error and the request is retried as defined in the MeteoFranceClient.

If the data were to be unavailable still, the order_id is stored as client._order_id. It can be used to request the data later, with client._retrieve_file(client._order_id)

# Exploring available stations

The other methods in DPClim are designed to help you explore the available stations and their characteristics, in order to identify the station code to use in fetch_data.

get_stations should be your first stop. It returns all stations from a departement (identified by its number). Each station is a dataframe with its basic characteristics, and in particular its 'id' used in fetch_data.

In [ ]:
# the result of client.get_stations is cached and stored in the client._stations dict.
stations = client.get_stations(1)  # or client.get_stations("01") or client.get_stations("1")

print("First station in the departement:")
print(stations[0])

# Options of get_stations:
stations = client.get_stations(
    1,
    lat=46.20426602709911,  # if lat, lon are given, stations are sorted by distance
    lon=5.2187206834000435,
    add_neighbours=False,  # add neighbouring departements, default True
    open_only=False,
)  # only return stations that are currently active, default True

print("Closes station to the given coordinates:")
print("Notice how it is different from the previous call")
print(stations[0])

The get_station_infom method provides more detailed information about a station, including its geographical coordinates, the parameters it observes, and its observation history. It can be used to check if a station has the parameters you are interested in, and if it has observations for the date range you want to fetch. It is, as exepected, called with the id of the station, and returns a dict.

In [ ]:
info = client.get_station_info(stations[0]["id"])

print("Keys of the info dict:")
print(info.keys())